<a href="https://colab.research.google.com/github/KarthikAmuthuri/MongoDB/blob/main/Students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install and start server
!apt install mongodb >log
!service mongodb start



 * Starting database mongodb
   ...done.


In [2]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names()

['admin', 'local']

In [3]:
#creating database called "school"
mydb = client['school'] #or client.school

In [4]:
#creating collection called students
mycol = mydb.students

In [5]:
client.list_database_names()

['admin', 'local']

In [6]:
mydb.list_collection_names()

[]

In [7]:
import json
data = [json.loads(line) for line in open('students.json', 'r' )]
len(data)

200

In [8]:
j = 0
for i in range(1,200):
  mydb.mycol.insert_many(data[j:i])
  j += 1

In [9]:
client.list_database_names()

['admin', 'local', 'school']

In [10]:
mydb.list_collection_names()

['mycol']

In [11]:
mydb.mycol.find_one()

{'_id': 0,
 'name': 'aimee Zank',
 'scores': [{'score': 1.463179736705023, 'type': 'exam'},
  {'score': 11.78273309957772, 'type': 'quiz'},
  {'score': 35.8740349954354, 'type': 'homework'}]}

#1) Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [12]:
agg_result = mydb.mycol.aggregate([{'$unwind':'$scores'},{'$group': {'_id':'$_id','total_score':{'$sum':'$scores.score'}}},
                      {'$sort':{'max_score': -1}},{'$limit':1}])
for i in agg_result:
  print(i)

{'_id': 94, 'total_score': 155.7771022790884}


In [13]:
final_res = mydb.mycol.find({'_id':13},{'name'})
for i in final_res:
  print(i['name'])

Jessika Dagenais


The student who scored maximum in all is **Jessika Dagenais**

#2) Find students who scored below average in the exam and pass mark is 40%?

In [14]:
agg_res = mydb.mycol.aggregate([
    {'$unwind': '$scores'},
    {'$match' : {'scores.type': 'exam'}},
    {'$group': {
        '_id': '$_id',
        'avg_exam_score': {'$avg':'$scores.score'},
        'pass_mark' :{'$sum':{'$cond':[{'$gte' : ['$scores.score', 40]}, 1, 0]}},
        'below_avg': {'$sum': { "$cond": [{ "$lt": ["$scores.score", 48.50]}, 1, 0 ]}}}
        }
       ])

indices_of_below_avg_students_in_exam = []

for i in agg_res:
  if i['below_avg'] == 1 and i['pass_mark'] == 1:
    indices_of_below_avg_students_in_exam.append(i['_id'])

In [15]:
print("students who scored below average and passed in the exam:\n")
for i in indices_of_below_avg_students_in_exam:
  res = mydb.mycol.find_one({'_id': i})
  print(res)

students who scored below average and passed in the exam:

{'_id': 89, 'name': 'Cassi Heal', 'scores': [{'score': 43.04310994985133, 'type': 'exam'}, {'score': 0.006247360551892012, 'type': 'quiz'}, {'score': 63.88558436723092, 'type': 'homework'}]}
{'_id': 68, 'name': 'Jenise Mcguffie', 'scores': [{'score': 40.15210496060384, 'type': 'exam'}, {'score': 90.60219950183566, 'type': 'quiz'}, {'score': 51.58720341010564, 'type': 'homework'}]}
{'_id': 42, 'name': 'Kayce Kenyon', 'scores': [{'score': 44.62441703708117, 'type': 'exam'}, {'score': 27.38208798553111, 'type': 'quiz'}, {'score': 97.43587143437509, 'type': 'homework'}]}
{'_id': 155, 'name': 'Aleida Elsass', 'scores': [{'score': 42.89558347656537, 'type': 'exam'}, {'score': 94.10647660402866, 'type': 'quiz'}, {'score': 30.56402201379193, 'type': 'homework'}]}
{'_id': 5, 'name': 'Wilburn Spiess', 'scores': [{'score': 44.87186330181261, 'type': 'exam'}, {'score': 25.72395114668016, 'type': 'quiz'}, {'score': 63.42288310628662, 'type'

#3) Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.

In [16]:
query3 = mydb.mycol.aggregate([
    {'$unwind':'$scores'},
    {'$match':{'$or':[{'scores.type': 'exam'},{'scores.type':'quiz'},{'scores.type':'homework'}]}},
    {'$group':{
     '_id':'$_id',
     'total':{'$sum':'$scores.score'},
     'average':{'$avg':{'$sum':'$scores.score'}},
     }}])

for i in query3:
  if i['total'] > 200 or i['average'] >= 73:
    i['result'] = 'pass'
  else:
    i['result'] = 'fail'
  print(i)

{'_id': 94, 'total': 155.7771022790884, 'average': 51.925700759696134, 'result': 'fail'}
{'_id': 78, 'total': 116.94520656066045, 'average': 38.98173552022015, 'result': 'fail'}
{'_id': 120, 'total': 107.87159055374062, 'average': 35.95719685124687, 'result': 'fail'}
{'_id': 138, 'total': 115.54875333859893, 'average': 38.51625111286631, 'result': 'fail'}
{'_id': 36, 'total': 219.003552388691, 'average': 73.00118412956367, 'result': 'pass'}
{'_id': 189, 'total': 181.79622614209114, 'average': 60.598742047363714, 'result': 'fail'}
{'_id': 89, 'total': 106.93494167763413, 'average': 35.644980559211376, 'result': 'fail'}
{'_id': 146, 'total': 201.4156031244819, 'average': 67.1385343748273, 'result': 'pass'}
{'_id': 58, 'total': 120.66923159080469, 'average': 40.223077196934895, 'result': 'fail'}
{'_id': 72, 'total': 206.92216095602325, 'average': 68.97405365200775, 'result': 'pass'}
{'_id': 180, 'total': 160.22762084530626, 'average': 53.40920694843542, 'result': 'fail'}
{'_id': 70, 'tota

#4) Find the total and average of the exam, quiz and homework and store them in a separate collection. 

In [17]:
mydb.list_collection_names()

['mycol']

In [18]:
total_query = mydb.mycol.aggregate([{'$unwind':'$scores'},{'$group': {'_id':'$_id','total_score':{'$sum':'$scores.score'}}},
                      {'$sort':{'max_score': -1}}])

total_list = []
for i in total_query:
  total_list.append(i)

In [19]:
print('Total scores of the students in all categories:\n')
for i in range(199):
  print(total_list[i])

Total scores of the students in all categories:

{'_id': 94, 'total_score': 155.7771022790884}
{'_id': 78, 'total_score': 116.94520656066045}
{'_id': 120, 'total_score': 107.87159055374062}
{'_id': 138, 'total_score': 115.54875333859893}
{'_id': 36, 'total_score': 219.003552388691}
{'_id': 189, 'total_score': 181.79622614209114}
{'_id': 89, 'total_score': 106.93494167763413}
{'_id': 146, 'total_score': 201.4156031244819}
{'_id': 58, 'total_score': 120.66923159080469}
{'_id': 72, 'total_score': 206.92216095602325}
{'_id': 180, 'total_score': 160.22762084530626}
{'_id': 70, 'total_score': 176.07490979439646}
{'_id': 69, 'total_score': 202.4848718127078}
{'_id': 114, 'total_score': 130.9721943324432}
{'_id': 66, 'total_score': 94.82481083119796}
{'_id': 71, 'total_score': 178.913172052241}
{'_id': 27, 'total_score': 82.6560500090462}
{'_id': 113, 'total_score': 158.1297878869123}
{'_id': 63, 'total_score': 72.74345907466646}
{'_id': 54, 'total_score': 242.0364415445718}
{'_id': 149, 'tota

In [20]:
#inserting total scores into new colllection
total = mydb.total_scores
total.insert_many(total_list)

In [21]:
mydb.list_collection_names()

['mycol', 'total_scores']



---



In [22]:
avg_query = mydb.mycol.aggregate([
    {'$unwind':"$scores"},
    {'$match':{'$or':[{"scores.type": "exam"},{'scores.type':'quiz'},{'scores.type':'homework'}]}},
    {'$group':{
     '_id':'$_id',
     'average':{'$avg':{'$sum':'$scores.score'}}}}
     ])

avg_list = []
for i in avg_query:
  avg_list.append(i)

In [23]:
len(avg_list)

199

In [24]:
print('Average scores of the students in all categories:\n')
for i in range(199):
  print(avg_list[i])

Average scores of the students in all categories:

{'_id': 94, 'average': 51.925700759696134}
{'_id': 78, 'average': 38.98173552022015}
{'_id': 120, 'average': 35.95719685124687}
{'_id': 138, 'average': 38.51625111286631}
{'_id': 36, 'average': 73.00118412956367}
{'_id': 189, 'average': 60.598742047363714}
{'_id': 89, 'average': 35.644980559211376}
{'_id': 146, 'average': 67.1385343748273}
{'_id': 58, 'average': 40.223077196934895}
{'_id': 72, 'average': 68.97405365200775}
{'_id': 180, 'average': 53.40920694843542}
{'_id': 70, 'average': 58.691636598132156}
{'_id': 69, 'average': 67.4949572709026}
{'_id': 114, 'average': 43.6573981108144}
{'_id': 66, 'average': 31.608270277065987}
{'_id': 71, 'average': 59.63772401741367}
{'_id': 27, 'average': 27.552016669682065}
{'_id': 113, 'average': 52.70992929563744}
{'_id': 63, 'average': 24.247819691555488}
{'_id': 54, 'average': 80.6788138481906}
{'_id': 149, 'average': 47.16998485788662}
{'_id': 62, 'average': 44.09576148698297}
{'_id': 47, '

In [25]:
#inserting average scores into new colllection
mydb.averages.insert_many(avg_list)

In [26]:
mydb.list_collection_names()

['mycol', 'averages', 'total_scores']

#5) Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [73]:
query5 = mydb.mycol.aggregate([
    {'$unwind': '$scores'},
    {'$match':{'$or':[{'scores.type': 'exam'},{'scores.type':'quiz'},{'scores.type':'homework'}]}},
    {'$group': {
        '_id': '$_id',
        'avg_score': {'$avg':'$scores.score'},
        'pass_mark' :{'$sum':{'$cond':[{'$gte' : ['$scores.score', 40]}, 1, 0]}},
        'below_avg': {'$sum': { "$cond": [{ "$lt": ["$scores.score", 55.17]}, 1, 0 ]}}}
        }
       ])

students_with_below_average_and_above_40_per = []
for i in query5:
  if i['pass_mark'] == 2 and i['below_avg'] == 2:
    students_with_below_average_and_above_40_per.append(i)

since there are no students who scored below average and above 40% in all the categories.

we just consider the students who scored above 40% and below average in two categories

In [75]:
students_with_below_average_and_above_40_per

[{'_id': 89, 'avg_score': 35.644980559211376, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 70, 'avg_score': 58.691636598132156, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 149, 'avg_score': 47.16998485788662, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 86, 'avg_score': 45.02620787680845, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 42, 'avg_score': 56.48079215232912, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 132, 'avg_score': 48.562424057799916, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 155, 'avg_score': 55.855360698128656, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 5, 'avg_score': 44.67289918492647, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 26, 'avg_score': 46.65456820906835, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 19, 'avg_score': 46.17779245857398, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 3, 'avg_score': 46.23570750744486, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 15, 'avg_score': 39.16669547058058, 'pass_mark': 2, 'below_avg': 2},
 {'_id': 125, 'avg_score': 45.678096343520195, 'pass_mark':

In [77]:
mydb.new_collection1.insert_many(students_with_below_average_and_above_40_per)

In [78]:
mydb.list_collection_names()

['mycol', 'averages', 'new_collection1', 'total_scores']

# 6) Create a new collection which consists of students who scored below the fail mark in all the categories.

In [89]:
query6 = mydb.mycol.aggregate([
    {'$unwind': '$scores'},
    {'$match':{'$or':[{'scores.type': 'exam'},{'scores.type':'quiz'},{'scores.type':'homework'}]}},
    {'$group': {
        '_id': '$_id',
        'passed_categories' :{'$sum':{'$cond':[{'$gte' : ['$scores.score', 40]}, 1, 0]}}}}
       ])

fail_list = []
for i in query6:
  if i['passed_categories'] == 0:
    fail_list.append(i)

In [92]:
fail_list

[{'_id': 0, 'passed_categories': 0}]

In [90]:
mydb.new_collecction2.insert_many(fail_list)

In [91]:
mydb.list_collection_names()

['mycol', 'averages', 'new_collecction2', 'new_collection1', 'total_scores']

#7) Create a new collection which consists of students who scored above pass mark in all the categories.

In [96]:
query7 = mydb.mycol.aggregate([
    {'$unwind': '$scores'},
    {'$match':{'$or':[{'scores.type': 'exam'},{'scores.type':'quiz'},{'scores.type':'homework'}]}},
    {'$group': {
        '_id': '$_id',
        'above_pass_mark' :{'$sum':{'$cond':[{'$gt' : ['$scores.score', 40]}, 1, 0]}}}}
       ])

above_pass_list =[]
for i in query7:
  if i['above_pass_mark'] == 3:
    above_pass_list.append(i)


In [100]:
above_pass_list

[{'_id': 36, 'above_pass_mark': 3},
 {'_id': 54, 'above_pass_mark': 3},
 {'_id': 68, 'above_pass_mark': 3},
 {'_id': 13, 'above_pass_mark': 3},
 {'_id': 18, 'above_pass_mark': 3},
 {'_id': 79, 'above_pass_mark': 3},
 {'_id': 64, 'above_pass_mark': 3},
 {'_id': 67, 'above_pass_mark': 3},
 {'_id': 11, 'above_pass_mark': 3},
 {'_id': 108, 'above_pass_mark': 3},
 {'_id': 90, 'above_pass_mark': 3},
 {'_id': 166, 'above_pass_mark': 3},
 {'_id': 61, 'above_pass_mark': 3},
 {'_id': 7, 'above_pass_mark': 3},
 {'_id': 85, 'above_pass_mark': 3},
 {'_id': 1, 'above_pass_mark': 3},
 {'_id': 55, 'above_pass_mark': 3},
 {'_id': 65, 'above_pass_mark': 3},
 {'_id': 23, 'above_pass_mark': 3},
 {'_id': 75, 'above_pass_mark': 3},
 {'_id': 52, 'above_pass_mark': 3},
 {'_id': 32, 'above_pass_mark': 3},
 {'_id': 84, 'above_pass_mark': 3},
 {'_id': 130, 'above_pass_mark': 3},
 {'_id': 98, 'above_pass_mark': 3},
 {'_id': 99, 'above_pass_mark': 3},
 {'_id': 102, 'above_pass_mark': 3},
 {'_id': 12, 'above_pass_m

In [102]:
mydb.new_collection3.insert_many(above_pass_list)

In [103]:
mydb.list_collection_names()

['new_collection3',
 'mycol',
 'averages',
 'new_collecction2',
 'new_collection1',
 'total_scores']